# Get Coverage

In [7]:
#!pip install mapclassify
from utils_spatial import *

In [8]:
# Country and Amenity
###########################
amenity = 'hospital'

# Transportation profile
###########################
profile='driving'

# Population of interest
###########################
population_ = 'women_of_reproductive_age_15_49'
minute = '15'

In [ ]:
isochrones_data = pd.read_csv(f"../data/LAC_{amenity}_{profile}.csv")
isochrones_data = isochrones_data[['isoalpha3', 'amenity', 'profile', 'minutes', 'multipolygon']]
isochrones_data = isochrones_data[~(isochrones_data['multipolygon'].isna())]


isochrone = isochrones_data[isochrones_data.minutes == int(minute)].reset_index()
isochrone = isochrone[~(isochrone.multipolygon.isna())].reset_index()
isochrone['geometry'] = gpd.GeoSeries.from_wkt(isochrone['multipolygon'])
geom = gpd.GeoDataFrame(isochrone, geometry='geometry')
geom = geom[['isoalpha3', 'amenity', 'profile', 'minutes', 'multipolygon', 'geometry']]
geom
###
#isochrone['geometry'] = gpd.GeoSeries.from_wkt(isochrone['geometry'])
#geom = geom[['isoalpha3', 'amenity', 'profile', 'minutes', 
#             'geometry']]

population = gpd.read_file(scldatalake + 
                           'Development Data Partnership/'+
                           'Facebook - High resolution population density map/'+
                           f'public-fb-data/geojson/LAC/{population_}_4'+
                           '.geojson')
population = population[~(population.isoalpha3.isna()) & (population.isoalpha3.isin(countries))]
population.population.sum()

output = []
# some isochrones cover more than one country
for isoalpha3 in countries:
    gdf_match_iso = sjoin(population[population.isoalpha3==isoalpha3], geom[geom.isoalpha3==isoalpha3], how='left')
    output.append(gdf_match_iso)
# ToDo(rsanchezavalos) Check inconsistent 
# gdf_match.population.sum()
gdf_match = pd.concat(output)
gdf_match.population.sum()
del gdf_match['index_right']
gdf_match.loc[(gdf_match.profile.isna()), 'profile'] = profile
gdf_match.loc[(gdf_match.amenity.isna()), f'coverage_{minute}'] = 'uncovered'
gdf_match.loc[~(gdf_match.amenity.isna()), f'coverage_{minute}'] = 'covered'
gdf_match.loc[(gdf_match.population.isna()), 'population'] = 0
gdf_match.loc[(gdf_match.minutes.isna()), 'minutes'] = minute
gdf_match.loc[(gdf_match.amenity.isna()), 'amenity'] = amenity
# log value column 
gdf_match['value'] = np.log(gdf_match.population)
gdf_match.loc[(gdf_match['value']<0),'value']=0

pd.options.display.float_format = '{:.2f}'.format
out = gdf_match.groupby(['amenity', f'coverage_{minute}'])['population'].sum().reset_index()
out['pct'] = out.population/out.population.sum()*100
out


level='2'
world = gpd.read_file(scldatalake +
                      'Geospatial Basemaps/Cartographic Boundary Files/world/level-1/world-level-1.zip')
world = world.rename(columns={"GID_0":"isoalpha3", 'NAME_1':'admin_name'})
world = world[(world.isoalpha3.isin(countries))]
world = world[(world.isoalpha3.isin(geom.isoalpha3))]


test = sjoin(world, gdf_match, how='left')
level_1 = test.groupby(['isoalpha3','admin_name', f'coverage_{minute}'])['population'].sum().reset_index()
t = level_1.groupby(['isoalpha3','admin_name'])['population'].sum().reset_index().rename(columns={'population':'poptot'})
level_1 = level_1.merge(t, on=['isoalpha3','admin_name'], how='left')
level_1['pct'] = level_1.population/level_1.poptot*100

level_1 = level_1[level_1.isoalpha3.isin(geom.isoalpha3)]

In [ ]:
%%capture
# Country and Amenity
###########################
amenity = 'hospital'

# Transportation profile
###########################
profile='driving'

# Population of interest
###########################
for population_ in ['women_of_reproductive_age_15_49', 'youth_15_24', 'elderly_60_plus']:
    for minute in ['15', '30', '45']:
        isochrones_data = pd.read_csv(f"../data/LAC_{amenity}_{profile}.csv")
        isochrones_data = isochrones_data[['isoalpha3', 'amenity', 'profile', 'minutes', 'multipolygon']]
        isochrones_data = isochrones_data[~(isochrones_data['multipolygon'].isna())]


        isochrone = isochrones_data[isochrones_data.minutes == int(minute)].reset_index()
        isochrone = isochrone[~(isochrone.multipolygon.isna())].reset_index()
        isochrone['geometry'] = gpd.GeoSeries.from_wkt(isochrone['multipolygon'])
        geom = gpd.GeoDataFrame(isochrone, geometry='geometry')
        geom = geom[['isoalpha3', 'amenity', 'profile', 'minutes', 'multipolygon', 'geometry']]
        geom
        ###
        #isochrone['geometry'] = gpd.GeoSeries.from_wkt(isochrone['geometry'])
        #geom = geom[['isoalpha3', 'amenity', 'profile', 'minutes', 
        #             'geometry']]

        population = gpd.read_file(scldatalake + 
                                   'Development Data Partnership/'+
                                   'Facebook - High resolution population density map/'+
                                   f'public-fb-data/geojson/LAC/{population_}_4'+
                                   '.geojson')
        population = population[~(population.isoalpha3.isna()) & (population.isoalpha3.isin(countries))]
        population.population.sum()

        output = []
        # some isochrones cover more than one country
        for isoalpha3 in countries:
            gdf_match_iso = sjoin(population[population.isoalpha3==isoalpha3], geom[geom.isoalpha3==isoalpha3], how='left')
            output.append(gdf_match_iso)
        # ToDo(rsanchezavalos) Check inconsistent 
        # gdf_match.population.sum()
        gdf_match = pd.concat(output)
        gdf_match.population.sum()
        del gdf_match['index_right']
        gdf_match.loc[(gdf_match.profile.isna()), 'profile'] = profile
        gdf_match.loc[(gdf_match.amenity.isna()), f'coverage_{minute}'] = 'uncovered'
        gdf_match.loc[~(gdf_match.amenity.isna()), f'coverage_{minute}'] = 'covered'
        gdf_match.loc[(gdf_match.population.isna()), 'population'] = 0
        gdf_match.loc[(gdf_match.minutes.isna()), 'minutes'] = minute
        gdf_match.loc[(gdf_match.amenity.isna()), 'amenity'] = amenity
        # log value column 
        gdf_match['value'] = np.log(gdf_match.population)
        gdf_match.loc[(gdf_match['value']<0),'value']=0

        pd.options.display.float_format = '{:.2f}'.format
        out = gdf_match.groupby(['amenity', f'coverage_{minute}'])['population'].sum().reset_index()
        out['pct'] = out.population/out.population.sum()*100
        out


        level='2'
        world = gpd.read_file(scldatalake +
                              'Geospatial Basemaps/Cartographic Boundary Files/world/level-1/world-level-1.zip')
        world = world.rename(columns={"GID_0":"isoalpha3", 'NAME_1':'admin_name'})
        world = world[(world.isoalpha3.isin(countries))]
        world = world[(world.isoalpha3.isin(geom.isoalpha3))]


        test = sjoin(world, gdf_match, how='left')
        level_1 = test.groupby(['isoalpha3','admin_name', f'coverage_{minute}'])['population'].sum().reset_index()
        t = level_1.groupby(['isoalpha3','admin_name'])['population'].sum().reset_index().rename(columns={'population':'poptot'})
        level_1 = level_1.merge(t, on=['isoalpha3','admin_name'], how='left')
        level_1['pct'] = level_1.population/level_1.poptot*100

        level_1 = level_1[level_1.isoalpha3.isin(geom.isoalpha3)]

        level_1 = level_1[level_1[f'coverage_{minute}']=='uncovered'].sort_values('population', ascending=False)
        level_1


        ax = sns.barplot(y="admin_name", x="population", data=level_1.head(10), palette="Blues_d")
        country_result = world.merge(level_1, on=['isoalpha3','admin_name'], how='left')
        fig, ax = plt.subplots(1, figsize=(10, 6))
        country_result.plot( figsize=(9, 9), column='pct', 
                            ax=ax, 
                            scheme='QUANTILES', k=20, 
                             cmap='BuPu', legend=True,
                             legend_kwds={'loc': 'center left', 'bbox_to_anchor':(1,0.5),  'fmt':"{:.0f}"})
        ax.set_title(f'Percentage of population without coverage - Level 1 {amenity} + {profile}' + 'min')

        country_result.to_csv(f'../data/coverage_{population_}_{minute}.csv',index=False)

In [4]:
data = pd.read_csv('../data/coverage_elderly_60_plus_15.csv')